## Day 8: Seven Segment Search
[link](https://adventofcode.com/2021/day/8)

### Parsing

In [ ]:
#load "../common.fsx"

module Segments = 
    type T = private Segments of Set<char> with
        static member (/) ((Segments s1), (Segments s2)) =
            Segments (Set.difference s1 s2)
        static member (+) ((Segments s1), (Segments s2)) =
            Segments (Set.union s2 s1)
        override this.ToString() = 
            let (Segments chars) = this
            new string(Set.toArray chars)
    let create s = 
        Segments (s |> Set.ofSeq)
    let apply f (Segments chars) = f chars
    let value seg = apply id seg
    let count = apply Set.count
    let contains (Segments sub) (Segments super) =
        Set.isSubset sub super


In [ ]:
let private parseSegmentsLine (s : string) = 
    s.Split(" ")
    |> Array.map Segments.create
let parse (input : string) = 
    input.Split([|"\n"; "\r"|], StringSplitOptions.RemoveEmptyEntries)
    |> Array.map (fun line -> splitToTuple2 [|" | "|] line)
    |> Array.map (fun (patterns, output) -> parseSegmentsLine patterns, parseSegmentsLine output)

In [ ]:
#!value --name sampleRaw --from-file ./sample
#!value --name inputRaw --from-file ./data

In [ ]:
#!share sampleRaw --from value
#!share inputRaw --from value
let sampleInput = parse sampleRaw
let actualInput = parse inputRaw
sampleInput

index,Item1,Item2
0,"[ be, abcdefg, bcdefg, acdefg, bceg, cdefg, abdefg, bcdef, abcdf, bde ]","[ abcdefg, bcdef, bcdefg, bceg ]"
1,"[ abdefg, bcdeg, bcg, cg, abcdefg, bdefg, abcdfg, abcde, bcdefg, cefg ]","[ bcdefg, bcg, abcdefg, cg ]"
2,"[ abdefg, cg, abcde, abdfg, abcdfg, bcdefg, abcdg, acfg, bcg, abcdefg ]","[ cg, cg, abcdfg, bcg ]"
3,"[ bcdefg, bcd, abcdef, abdeg, abcf, bc, acdef, abcde, acdefg, abcdefg ]","[ abcdef, abcde, acdefg, bc ]"
4,"[ abcdefg, bfg, fg, abefg, abdef, cefg, abceg, abcefg, abcdeg, abcdfg ]","[ cefg, abcdefg, bfg, abefg ]"
5,"[ abefg, ac, abcefg, abcdefg, acdefg, bcdfg, abce, abdefg, abcfg, acf ]","[ abcdefg, abce, ac, abcdefg ]"
6,"[ bcdfg, dfg, abcdefg, cefg, abdefg, abcdef, bcdef, abcdg, bcdefg, fg ]","[ cefg, bcdef, cefg, abcdefg ]"
7,"[ bcdefg, abcefg, bcefg, acdefg, abcdg, de, bdef, cde, abcdefg, bcdeg ]","[ de, abcefg, abcdg, bcefg ]"
8,"[ abdefg, bcdefg, cdeg, abcef, bcg, abcdefg, cg, abcdfg, bdefg, bcefg ]","[ abcdefg, bcg, cg, bcg ]"
9,"[ abcfg, cfg, abcdefg, abceg, fg, abcdeg, aefg, abcefg, abcdf, bcdefg ]","[ aefg, abcfg, fg, abceg ]"


### Part 1

In [ ]:

let part1 input = 
    input
    |> Array.collect snd
    |> Array.filter (fun s -> match (Set.count (Segments.value s)) with | 2 | 4 | 3 | 7 -> true | _ -> false)
    |> Array.length

sampleInput |> part1 |> display
actualInput |> part1 |> display

26

532

### Part 2

The main piece of information using which we can differentiate digits is segments count. Next, it is beneficial to consider digits with the same amount together.

```
2 segments:
 ....  
.    c 
.    c 
 ....  
.    f 
.    f 
 ....  

3 segments:
 aaaa  
.    c 
.    c 
 ....  
.    f 
.    f 
 ....  

4 segments:
 ....
b    c
b    c
 dddd
.    f
.    f
 ....

7 segments:
 aaaa  
b    c 
b    c 
 dddd  
e    f 
e    f 
 gggg  
```

So, we can identify digits `1`, `4`, `7`, `8` only by segments count.

For the rest digits:

```
5 segments:
 aaaa      aaaa      aaaa  
.    c    .    c    b    . 
.    c    .    c    b    . 
 dddd      dddd      dddd  
e    .    .    f    .    f 
e    .    .    f    .    f 
 gggg      gggg      gggg  

6 segments:
 aaaa      aaaa      aaaa
b    c    b    .    b    c
b    c    b    .    b    c
 ....      dddd      dddd
e    f    e    f    .    f
e    f    e    f    .    f
 gggg      gggg      gggg
```

Here, we can see that, for instance, digit `3` is the only in its group which contains digit `1` in it.\
Digit `5` is the only in its group which contains segments of `4 / 7` in it.\
Digit `2` is the only in its group which contains segments of `8 / 4` in it.\
Digit `9` is the only in its group which contains digit `4` in it.\
Digit `6` is the only in its group which does not contain digit `7` in it.\
Digit `0` is the only in its group which contains segments of `8 / 4 + 1` in it.



In [ ]:
let resolveDigits (segments: Segments.T array) = 
    let byLen = segments |> Array.groupBy Segments.count |> Map.ofArray
    let ``1`` = byLen.[2] |> Array.exactlyOne
    let ``4`` = byLen.[4] |> Array.exactlyOne
    let ``7`` = byLen.[3] |> Array.exactlyOne
    let ``8`` = byLen.[7] |> Array.exactlyOne
    let ``3`` = byLen.[5] |> Array.filter (Segments.contains ``1``) |> Array.exactlyOne
    let ``5`` = byLen.[5] |> Array.filter (Segments.contains (``4`` / ``7``)) |> Array.exactlyOne
    let ``2`` = byLen.[5] |> Array.filter (Segments.contains (``8`` / ``4``)) |> Array.exactlyOne
    let ``9`` = byLen.[6] |> Array.filter (Segments.contains ``4``) |> Array.exactlyOne
    let ``6`` = byLen.[6] |> Array.filter (not << Segments.contains ``7``) |> Array.exactlyOne
    let ``0`` = byLen.[6] |> Array.filter (Segments.contains (``8`` / ``4`` + ``1``)) |> Array.exactlyOne
    [
        ``0``, 0
        ``1``, 1
        ``2``, 2
        ``3``, 3
        ``4``, 4
        ``5``, 5
        ``6``, 6
        ``7``, 7
        ``8``, 8
        ``9``, 9
    ] |> Map.ofList

let calculate (segments: Segments.T array) (segmentsMap: Map<Segments.T, int>) = 
    let len = Array.length segments
    segments 
    |> Array.mapi (fun i seg -> segmentsMap.[seg] * (pown 10 (len - 1 - i)))
    |> Array.sum


In [ ]:
let part2 input = 
    input 
    |> Array.map (fun (patterns, output) -> calculate output (resolveDigits patterns))
    |> Array.sum

sampleInput |> part2 |> display
actualInput |> part2 |> display

61229

1011284